In [ ]:
!pip install transformers datasets evaluate sacrebleu tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

GPU Available: True
Current Device: 0
Device Name: Tesla T4


In [ ]:
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.32.2
    Uninstalling huggingface-hub-0.32.2:
      Successfully uninstalled huggingface-hub-0.32.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.

In [ ]:
from huggingface_hub import notebook_login

# Login to Huggingface
notebook_login()

In [ ]:
# Load OPUS books dataset
from datasets import load_dataset

books = load_dataset("sentence-transformers/parallel-sentences-opensubtitles", "en-es")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

en-es/train-00000-of-00003.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

en-es/train-00001-of-00003.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

en-es/train-00002-of-00003.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13646844 [00:00<?, ? examples/s]

In [ ]:
#books1 = load_dataset("opus_books", "en-es")

README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/93470 [00:00<?, ? examples/s]

In [ ]:
books["train"] = books["train"].select(range(100000))

In [ ]:
# Split the dataset
books = books["train"].train_test_split(test_size=0.1)

In [ ]:
books["test"][1]

{'english': "It swallowed it whole. You could see the crocodile in the anaconda's tummy. We were speechless...",
 'non_english': 'ya que apareció el cocodrilo en el abdomen, quedó solo, mira'}

In [ ]:
# Load T5 tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
#from transformers import T5Tokenizer, T5ForConditionalGeneration
checkpoint = "google-t5/t5-small"
#checkpoint = "arnauad/my_model_f"
#checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
source_lang = "english"
target_lang = "non_english"
# Prompt that will be added to the beginning of each input sentence
prefix = "translate English to Spanish: "

def preprocess_function(examples):
    inputs = [prefix + src for src in examples[source_lang]]
    targets = [tgt for tgt in examples[target_lang]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(targets, max_length=512, truncation=True).input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Apply the processing function over the entire dataset
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load T5 model
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

# Prepare the data for a sequence to sequence model
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
import evaluate

# Include a metric during training to evaluate our model performance
metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np

# Takes the predictions and the labels and removes leading and trailing whitespace
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

# Calculate evaluation metrics
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Handle the scenario where preds might be a tuple itself
    if isinstance(preds, tuple):
        preds = preds[0]

    # Decode back to human readable text
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Same, but values of -100 indicate ignored tokens
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute the BLEU score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    # Computes the average length of the generated predictions
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
for param in model.encoder.parameters():
    param.requires_grad = False

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

shared.weight
encoder.block.0.layer.0.SelfAttention.q.weight
encoder.block.0.layer.0.SelfAttention.k.weight
encoder.block.0.layer.0.SelfAttention.v.weight
encoder.block.0.layer.0.SelfAttention.o.weight
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
encoder.block.0.layer.0.layer_norm.weight
encoder.block.0.layer.1.DenseReluDense.wi.weight
encoder.block.0.layer.1.DenseReluDense.wo.weight
encoder.block.0.layer.1.layer_norm.weight
encoder.block.1.layer.0.SelfAttention.q.weight
encoder.block.1.layer.0.SelfAttention.k.weight
encoder.block.1.layer.0.SelfAttention.v.weight
encoder.block.1.layer.0.SelfAttention.o.weight
encoder.block.1.layer.0.layer_norm.weight
encoder.block.1.layer.1.DenseReluDense.wi.weight
encoder.block.1.layer.1.DenseReluDense.wo.weight
encoder.block.1.layer.1.layer_norm.weight
encoder.block.2.layer.0.SelfAttention.q.weight
encoder.block.2.layer.0.SelfAttention.k.weight
encoder.block.2.layer.0.SelfAttention.v.weight
encoder.block.2.layer.0.SelfAttentio

In [ ]:
from transformers import TrainerCallback

class PrintMetricsCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is not None:
            print(f"\n Epoch {int(state.epoch)} Evaluation Metrics:")
            for key, value in metrics.items():
                print(f"  {key}: {value:.4f}")

In [ ]:
# Define our hyperparameters
training_args = Seq2SeqTrainingArguments(
    output_dir = "my_model_mt5",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 5e-5,
    lr_scheduler_type="cosine",
    warmup_steps=500,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    num_train_epochs = 5,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=50,
    report_to="tensorboard",
)

# Pass the training arguments
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_books["train"],
    eval_dataset = tokenized_books["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    callbacks=[PrintMetricsCallback()],
)

trainer.train()

trainer.push_to_hub()

<ipython-input-33-eaa92f4dee0d>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.000000,nan,0.000000,0.000000
2,0.000000,nan,0.000000,0.000000



 Epoch 1 Evaluation Metrics:
  eval_loss: nan
  eval_bleu: 0.0000
  eval_gen_len: 0.0000
  eval_runtime: 330.4304
  eval_samples_per_second: 30.2640
  eval_steps_per_second: 1.8910
  epoch: 1.0000

 Epoch 2 Evaluation Metrics:
  eval_loss: nan
  eval_bleu: 0.0000
  eval_gen_len: 0.0000
  eval_runtime: 329.6910
  eval_samples_per_second: 30.3310
  eval_steps_per_second: 1.8960
  epoch: 2.0000


KeyboardInterrupt: 

## TEST

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
import torch

repo_name = "arnauad/my_model_f"

tokenizer = AutoTokenizer.from_pretrained(repo_name)
model  = AutoModelForSeq2SeqLM.from_pretrained(repo_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

translator = pipeline(
    "translation_en_to_es",
    model=model,
    tokenizer=tokenizer,
    device=0 if device=="cuda" else -1,
)

examples = [
    "Hello, how are you?",
    "Transformers are amazing for machine translation.",
    "The weather today is perfect for a walk."
]
for src in examples:
    out = translator(src, max_length=128)[0]["translation_text"]
    print(f"EN: {src}\nES: {out}\n")


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


EN: Hello, how are you?
ES: Hol, cóm t?



Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


EN: Transformers are amazing for machine translation.
ES: Transformers son encantador para la traducción de las robos.

EN: The weather today is perfect for a walk.
ES: Ahora es perfect para un pase.

